In [16]:
import pandas as pd
from datasets import Dataset

df = pd.read_csv("train_model_data/wiki_dataset.csv", sep=";")

dataset = Dataset.from_pandas(df)

dataset = dataset.train_test_split(test_size=0.2, seed=42)
train_dataset = dataset["train"]
eval_dataset = dataset["test"]


In [17]:
from transformers import AutoTokenizer

# model_name = "google/mt5-small"
model_name = "facebook/mbart-large-50"
tokenizer = AutoTokenizer.from_pretrained(model_name)

def preprocess(examples):
    inputs = examples["text_no_spaces"]
    targets = examples["text_restored"]

    model_inputs = tokenizer(inputs, truncation=True, padding=True, max_length=32)
    labels = tokenizer(targets, truncation=True, padding=True, max_length=32)

    model_inputs["labels"] = [
        [(l if l != tokenizer.pad_token_id else -100) for l in label]
        for label in labels["input_ids"]
    ]

    return model_inputs


train_dataset = train_dataset.map(preprocess, batched=True)
eval_dataset = eval_dataset.map(preprocess, batched=True)

c:\Work_life\HSE_and_study\Intern\Avito\venv\Lib\site-packages\huggingface_hub\file_download.py:143: UserWarning: `huggingface_hub` cache-system uses symlinks by default to efficiently store duplicated files but your machine does not support them in C:\Users\Max\.cache\huggingface\hub\models--facebook--mbart-large-50. Caching files will still work but in a degraded version that might require more space on your disk. This warning can be disabled by setting the `HF_HUB_DISABLE_SYMLINKS_WARNING` environment variable. For more details, see https://huggingface.co/docs/huggingface_hub/how-to-cache#limitations.
To support symlinks on Windows, you either need to activate Developer Mode or to run Python as an administrator. In order to activate developer mode, see this article: https://docs.microsoft.com/en-us/windows/apps/get-started/enable-your-device-for-development
  warnings.warn(message)
Xet Storage is enabled for this repo, but the 'hf_xet' package is not installed. Falling back to regul

### Токенизация и подготовка датасета

1. **Загрузка токенизатора**  
   Используем токенизатор модели `facebook/mbart-large-50`.

2. **Функция `preprocess`**  
   - Вход: примеры с текстом без пробелов (`text_no_spaces`) и восстановленным текстом (`text_restored`).  
   - Токенизирует входной и целевой текст с усечением до `max_length=32` и паддингом.  
   - Заменяет паддинг токены на `-100` в метках, чтобы лосс-функция их игнорировала.

3. **Применение к датасету**  
   - `train_dataset.map(preprocess, batched=True)` — токенизация обучающего датасета.  
   - `eval_dataset.map(preprocess, batched=True)` — токенизация валидационного датасета.

> В результате получаем токенизированный датасет, готовый для подачи в модель Seq2Seq.


In [ ]:
from transformers import AutoModelForSeq2SeqLM, DataCollatorForSeq2Seq, TrainingArguments, Trainer
import torch

model_name = "facebook/mbart-large-50"
model = AutoModelForSeq2SeqLM.from_pretrained(model_name).to(torch.device("cuda" if torch.cuda.is_available() else "cpu"))
data_collator = DataCollatorForSeq2Seq(tokenizer, model=model)

training_args = TrainingArguments(
    output_dir="./mbart-space-fix",
    num_train_epochs=2,
    per_device_train_batch_size=16,
    per_device_eval_batch_size=16,
    learning_rate=1e-4,
    weight_decay=0.01,
    logging_dir="./logs",
    logging_steps=50,
    save_steps=500,
    save_total_limit=2,
    fp16=True,
    eval_steps=50
)

trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=train_dataset,
    eval_dataset=eval_dataset,
    processing_class=tokenizer,
    data_collator=data_collator
)

Xet Storage is enabled for this repo, but the 'hf_xet' package is not installed. Falling back to regular HTTP download. For better performance, install the package with: `pip install huggingface_hub[hf_xet]` or `pip install hf_xet`
Error while downloading from https://huggingface.co/facebook/mbart-large-50/resolve/refs%2Fpr%2F7/model.safetensors: HTTPSConnectionPool(host='cas-bridge.xethub.hf.co', port=443): Read timed out.
Trying to resume download...


### Настройка и обучение модели Seq2Seq

1. **Загрузка модели**  
   Используется `facebook/mbart-large-50`. Модель сразу переводится на GPU (если доступен).

2. **DataCollator**  
   - `DataCollatorForSeq2Seq` автоматически подготавливает батчи, выравнивая длины входов и меток.

3. **TrainingArguments**  
   - `output_dir` — папка для чекпоинтов.  
   - `num_train_epochs` — количество эпох обучения.  
   - `per_device_train_batch_size`, `per_device_eval_batch_size` — размеры батчей.  
   - `learning_rate`, `weight_decay` — параметры оптимизации.  
   - `logging_dir`, `logging_steps` — логирование.  
   - `save_steps`, `save_total_limit` — сохранение чекпоинтов.  
   - `fp16=True` — использование смешанной точности (если есть поддержка GPU).  
   - `eval_steps` — частота оценки на валидации.

4. **Trainer**  
   - Инкапсулирует весь процесс обучения: подачу батчей, вычисление лосса, оптимизацию, логирование и сохранение модели.  
   - `train_dataset` и `eval_dataset` — подготовленные токенизированные датасеты.  
   - `tokenizer` используется для обработки текстов.  
   - `data_collator` выравнивает батчи по длине.


In [22]:
trainer.train()

Step,Training Loss
50,5.496000
100,0.774100
150,0.592200
200,0.595600
250,0.517600
300,0.493600
350,0.373000
400,0.414300
450,0.418300
500,0.320000


c:\Work_life\HSE_and_study\Intern\Avito\venv\Lib\site-packages\transformers\modeling_utils.py:4037: UserWarning: Moving the following attributes in the config to the generation config: {'max_length': 200, 'early_stopping': True, 'num_beams': 5}. You are seeing this warning because you've set generation parameters in the model config, as opposed to in the generation config.
  warnings.warn(


TrainOutput(global_step=1300, training_loss=0.45927350007570705, metrics={'train_runtime': 4918.9503, 'train_samples_per_second': 4.229, 'train_steps_per_second': 0.264, 'total_flos': 904912123723776.0, 'train_loss': 0.45927350007570705, 'epoch': 2.0})

In [ ]:
import torch

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

text = "Нонадорогетыегоненайдешь"

inputs = tokenizer(text, return_tensors="pt").to(device)

model.to(device)

output_ids = model.generate(**inputs, max_length=50)

print(tokenizer.decode(output_ids[0], skip_special_tokens=True))

Но надорогеты его не будешь


In [24]:
inputs = tokenizer(text, return_tensors="pt").to(device)
model.to(device)

# Генерация с beam search
output_ids = model.generate(
    **inputs,
    max_length=50,
    num_beams=5,
    early_stopping=True
)

decoded = tokenizer.decode(output_ids[0], skip_special_tokens=True)
print(decoded)


Москва 2023 телефон Philips


In [ ]:
import torch
print(torch.version.cuda)          
print(torch.cuda.is_available())   
print(torch.cuda.get_device_name(0))  

12.6
True
NVIDIA GeForce RTX 3060 Laptop GPU


Выгрузка в HuggingFace

In [ ]:
from huggingface_hub import login

login(token="")

In [29]:
model.push_to_hub("Dropdead072/mbart-large-50-spaces", use_auth_token=True)

model.safetensors: 100%|██████████| 2.44G/2.44G [08:24<00:00, 4.84MB/s]   


CommitInfo(commit_url='https://huggingface.co/Dropdead072/mbart-large-50-spaces/commit/7d8e7505dc080589dc10e21e97b41980d710f885', commit_message='Upload MBartForConditionalGeneration', commit_description='', oid='7d8e7505dc080589dc10e21e97b41980d710f885', pr_url=None, repo_url=RepoUrl('https://huggingface.co/Dropdead072/mbart-large-50-spaces', endpoint='https://huggingface.co', repo_type='model', repo_id='Dropdead072/mbart-large-50-spaces'), pr_revision=None, pr_num=None)